# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언

인자(argument)의 역할과 내용
- pretrained_model_name : 이전 장에서 파인튜닝한 모델이 사용한 프리트레인 마친 언어모델 이름(단 해당 모델은 허깅페이스 라이브러리에 등록되어 있어야 합니다)
- downstream_model_dir : 이전 장에서 파인튜닝한 모델의 체크포인트 저장 위치.
- max_seq_length : 토큰 기준 입력 문장 최대 길이. 아무 것도 입력하지 않으면 128입니다.

In [ ]:
# TPU 관련 패키지 설치

!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pythorch
/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
# 의존성 패키지 설치
!pip install ratsnlp

In [ ]:
# 구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [5]:
# 인퍼런스 설정
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/content/drive/MyDrive/nlpbook/checkpoint-doccls1-20230726T003135Z-002",
    max_seq_length=128,
)


Exception: ignored

# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

- fine_tuned_model_ckpt['state_dict']: 파인튜닝된 모델의 상태를 가지고 있는 딕셔너리. 모델의 각 레이어에 대한 가중치와 bias 값들 저장
- 'model.classifier.bias': 분류기의 bias 텐서를 가져온다.이 분류기는 각 클래스에 대한 확률을 출력하며, bias 텐서 크기는 분류 클래스 수와 동일
- .shape.numel(): 텐서의 모든 원소의 개수를 반환. 이 경우에는 bias 텐서의 원소의 개수, 즉 클래스의 수를 반환

# 인퍼런스 함수 선언
- 문장(sentence)에 토큰화를 수행한 뒤 input_ids, attention_mask, token_type_ids를 만든다.
- 이들 입력값을 파이토치 텐서(tensor) 자료형으로 변환한 뒤 모델에 입력합니다. - 모델 출력 값(outputs.logits)은 소프트맥스 함수 적용 이전의 로짓(logit) 형태인데요. 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률] 형태의 확률 형태로 바꾼다.
- 마지막으로 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 경우 해당 문장이 부정(positive), 반대의 경우 긍정(positive)이 되도록 pred 값을 만든다.

In [6]:
!mkdir/root/.ngrok2 && echo "authtoken: 2T5dfeTEp7iYvvXPnicOeBb78lp_7f3UkTB35MjeK5T4Z31rw" > /root/.ngrok2/ngrok.yml   # 인증토큰

/bin/bash: line 1: mkdir/root/.ngrok2: No such file or directory
